In [8]:
corpus = open('corpus.txt').read()
corpus

'Валерий Медведев. Баранкин, будь человеком! (1957)   [омонимия снята] Все примеры (1)\nЯ уже говорил, что раньше нам тоже приходилось получать двойки, и не раз, но никогда ещё наши ребята не брали нас с Костей в самом начале года в такой оборот, как в эту субботу. [Валерий Медведев. Баранкин, будь человеком! (1957)] [омонимия снята] ←…→\nСергей Козлов. Правда, мы будем всегда? (1969-1981)   [омонимия снята] Все примеры (1)\nИ на обороте Ослиной берёсты он тщательно нацарапал: [Сергей Козлов. Правда, мы будем всегда? (1969-1981)] [омонимия снята] ←…→\nРуководство по эксплуатации велосипеда (1978)   [омонимия снята] Все примеры (4)\nДля установки руля на требуемую высоту необходимо повернуть на 3-4 оборота рукоятку зажима (1), ослабив хомутик (5) на стержне вилки, установить руль (2) на требуемую высоту и вновь закрепить. [Руководство по эксплуатации велосипеда (1978)] [омонимия снята] ←…→\nМихаил Задорнов. Приказ министра финансов РФ Об утверждении положения по бухгалтерскому учету (19

In [4]:
import re

In [14]:
reg_sent = re.compile('(.*?)\[.*?\] \[омонимия')

In [19]:
rows = reg_sent.findall(corpus)
rows[:5]

['Я уже говорил, что раньше нам тоже приходилось получать двойки, и не раз, но никогда ещё наши ребята не брали нас с Костей в самом начале года в такой оборот, как в эту субботу. ',
 'И на обороте Ослиной берёсты он тщательно нацарапал: ',
 'Для установки руля на требуемую высоту необходимо повернуть на 3-4 оборота рукоятку зажима (1), ослабив хомутик (5) на стержне вилки, установить руль (2) на требуемую высоту и вновь закрепить. ',
 'При этом события после отчётной даты отражаются в синтетическом и аналитическом учёте заключительными оборотами отчётного периода до утверждения годовой бухгалтерской отчётности в установленном порядке. ',
 'Была у меня форсированная шаха, с горки по прямой, при попутном ветре наверняка раскочегарил до 175 км / ч ― это как мне посчитали по оборотам движка (тахометр цифровой был) и передаточным числам. ']

In [1]:
import csv

In [24]:
with open('oborot.csv', 'w') as w:
    out = csv.writer(w, delimiter=',')
    for row in rows:
        out.writerow([row])

In [33]:
some = open('add.txt').read()

In [31]:
rows = reg_sent.findall(some)

In [32]:
with open('add.csv', 'w') as w:
    out = csv.writer(w, delimiter=',')
    for row in rows:
        out.writerow([row])

In [2]:
from pymystem3 import Mystem

In [5]:
reg_pos = re.compile('([A-Z].*?)[=,]')

In [6]:
m = Mystem()
test = []

In [29]:
with open('oborot.csv') as csvfile, \
    open('output_features.csv', 'w') as output:
    data = csv.reader(csvfile, delimiter=',')
    writer = csv.writer(output, delimiter=',')
    writer.writerow(['text','prevword', 'nextword', 'prevtag', 'nexttag', 'class'])
    for row in data:
        if row[0] != 'text':
            text = row[0]
            cl = row[1]
            ans = m.analyze(text)
            info = []
            for an in ans:
                try:
                    if an['analysis'] == []:
                        gram = 'UNKN'
                        lemma = an['text']
                    else:
                        analysis = an['analysis'][0]
                        #print(analysis)
                        gram = reg_pos.search(analysis['gr']).group(1)
                        lemma = analysis['lex']
                        #print(lemma, gram)
                    info.append([lemma, gram])
                except KeyError:
                    pass
            for i in range(len(info)):
                if info[i][0] == 'оборот':
                    try:
                        prevword = info[i-1][0]
                        prevtag = info[i-1][1]
                    except IndexError:
                        prevword = ''
                        prevtag = ''
                    try:
                        nextword = info[i+1][0]
                        nexttag = info[i+1][1]
                    except IndexError:
                        nextword = ''
                        nexttag = ''
            writer.writerow([text, prevword, nextword, prevtag, nexttag, cl])

{'analysis': [{'gr': 'SPRO,ед,1-л=им', 'lex': 'я'}], 'text': 'Я'}

In [13]:
test = 'сомнительное слово в пушкиноведческий оборот). Стихии как органические силы жизни , которым даёт язык поэзия. Вот вопрос понимания Пушкина: как оценить эти силы и роль их в "порядке мира" пушкинском, если вспомнить блоковское в его речи: "Порядок мира тревожен, он― родное дитя беспорядка…" 69? '

In [14]:
test

'сомнительное слово в пушкиноведческий оборот). Стихии как органические силы жизни , которым даёт язык поэзия. Вот вопрос понимания Пушкина: как оценить эти силы и роль их в "порядке мира" пушкинском, если вспомнить блоковское в его речи: "Порядок мира тревожен, он― родное дитя беспорядка…" 69? '

In [18]:
ans = m.analyze(test)

{'analysis': [{'gr': 'A=(вин,ед,полн,сред|им,ед,полн,сред)',
   'lex': 'сомнительный'}],
 'text': 'сомнительное'}

In [21]:
info = []

In [22]:
for an in ans:
    try:
        analysis = an['analysis'][0]
        #print(analysis)
        gram = analysis['gr']
        lemma = analysis['lex']
        info.append([lemma, reg_pos.search(gram).group(1)])
    except KeyError:
        pass
print(info)

[['сомнительный', 'A'], ['слово', 'S'], ['в', 'PR'], ['пушкиноведческий', 'A'], ['оборот', 'S'], ['стихия', 'S'], ['как', 'CONJ'], ['органический', 'A'], ['сила', 'S'], ['жизнь', 'S'], ['который', 'APRO'], ['давать', 'V'], ['язык', 'S'], ['поэзия', 'S'], ['вот', 'PART'], ['вопрос', 'S'], ['понимание', 'S'], ['пушкин', 'S'], ['как', 'CONJ'], ['оценивать', 'V'], ['этот', 'APRO'], ['сила', 'S'], ['и', 'CONJ'], ['роль', 'S'], ['они', 'SPRO'], ['в', 'PR'], ['порядок', 'S'], ['мир', 'S'], ['пушкинский', 'A'], ['если', 'CONJ'], ['вспомнить', 'V'], ['блоковский', 'A'], ['в', 'PR'], ['его', 'APRO'], ['речь', 'S'], ['порядок', 'S'], ['мир', 'S'], ['тревожный', 'A'], ['он', 'SPRO'], ['родной', 'A'], ['дитя', 'S'], ['беспорядок', 'S']]


In [25]:
for i in range(len(info)):
    if info[i][0] == 'оборот':
        try:
            prevword = info[i-1][0]
            prevtag = info[i-1][1]
        except IndexError:
            prevword = ''
            prevtag = ''
        try:
            nextword = info[i+1][0]
            nexttag = info[i+1][1]
        except IndexError:
            nextword = ''
            nexttag = ''
    

In [28]:
prevword, nextword

('пушкиноведческий', 'стихия')